# Example for using SCD Transformer

We demonstrate here how to use SCD Transformer for writing SCDs with your custom prompts.

In [ ]:
import os
from convokit import Corpus, download
from convokit.convo_similarity import SCD
from convokit.genai import GenAIConfigManager

In [ ]:
corpus = Corpus(filename=download("friends-corpus"))

In [ ]:
### Config your GenAI API keys
config = GenAIConfigManager()

In [ ]:
def format_friends_transcript_from_convokit(corpus, convo_id):
    convo = corpus.get_conversation(convo_id)
    utt_lst = convo.get_utterance_ids()
    speaker_ids = {}
    transcript = ""
    for utt_id in utt_lst:
        utt = corpus.get_utterance(utt_id)
        if "TRANSCRIPT_NOTE" not in utt.speaker.id:
            if utt.speaker.id not in speaker_ids:
                # print(utt.speaker.id, ":", utt.text)
                speaker_ids[utt.speaker.id] = 1 + len(speaker_ids)
            transcript += "Speaker"+str(speaker_ids[utt.speaker.id]) + " : " + utt.text+ "\n\n"
    return transcript

In [ ]:
friends_summary_prompt = """
Write a short summary capturing the trajectory of a casual conversation.
Do not include specific topics, events, or arguments from the conversation. The style you should avoid:
Example Sentence 1: “Speaker1 said they had a difficult day at work, and mentioned that their boss was unfair. Speaker2 listened and agreed that bosses can be tough, then suggested they go out for dinner to forget about it..”

Instead, do include indicators of sentiments (e.g., warmth, empathy, humor, nostalgia, vulnerability, support), individual intentions (e.g., building rapport, offering reassurance, seeking validation, self-disclosure, active listening, gentle disagreement, creating distance), and conversational strategies (if any) such as 'collaborative storytelling', 'inside jokes', 'mirroring emotions,' and 'affectionate teasing.'
The following sentences demonstrate the style you should follow:
* Example Sentence 2: “Both speakers have similar feelings and appeared mutually supportive. Speaker1 initiates with a moment of self-disclosure, and Speaker2 responds with empathy and validation. Both speakers build on this exchange, strengthening their rapport.”
* Example Sentence 3: “The two speakers connected with back-and-forth affectionate teasing. Throughout the conversation, they kept building on each other’s humor with playful remarks, creating a lighthearted and comfortable discussion.”

Overall, the trajectory summary should capture the key moments where the emotional connection of the conversation notably changes. Here is an example of a complete trajectory summary.
Trajectory Summary: The conversation begins with two speakers exchanging neutral, surface-level comments. Speaker1 then shifts the tone by sharing a personal anecdote, prompting Speaker2 to respond with warmth and empathy. Speaker1 elaborates on their story and their need, but Speaker2 does not extend their support but retracts it.
Now, provide the trajectory summary for the following conversation.
Conversation Transcript:{transcript}
Now, summarize this conversation. Remember, do not include specific topics, claims, or arguments from the conversation. Instead, try to capture the speakers' sentiments, intentions, and conversational/persuasive strategies. Limit the trajectory summary to 80 words.
Trajectory Summary:"""

Initialize your SCD transformer

In [ ]:
scd_transformer = SCD(
        model_provider="gpt",
        config=config,
        model="gpt-3.5-turbo",
        custom_scd_prompt=friends_summary_prompt,
        custom_prompt_dir="friends_prompts",
        generate_scd=True,
        generate_sop=True,
        scd_metadata_name="machine_scd",
        sop_metadata_name="machine_sop",
        conversation_formatter=format_friends_transcript_from_convokit
    )

In [ ]:
conversation_ids = list(corpus.get_conversation_ids())[:2]
selector = lambda conv: conv.id in conversation_ids

In [ ]:
corpus = scd_transformer.transform(corpus, selector=selector)

In [ ]:
convo = corpus.get_conversation(conversation_ids[0])
print("SCD: ", convo.meta["machine_scd"])
print("SoP: ", convo.meta["machine_sop"])